```
curl -fsSL https://ollama.com/install.sh | sh

ollama pull llama3.2:1b

pip install 'litellm[proxy]'
litellm --model ollama/llama3.2:1b

```

In [8]:
from dataclasses import dataclass

from autogen_core import (
    AgentId,
    DefaultTopicId,
    MessageContext,
    RoutedAgent,
    SingleThreadedAgentRuntime,
    default_subscription,
    message_handler,
)
from autogen_core.model_context import BufferedChatCompletionContext
from autogen_core.models import (
    AssistantMessage,
    ChatCompletionClient,
    SystemMessage,
    UserMessage,
)
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [ ]:
def get_model_client() -> OpenAIChatCompletionClient:  # type: ignore
    "Mimic OpenAI API using Local LLM Server."
    return OpenAIChatCompletionClient(
        model="ollama/llama3.2:1b",
        api_key="NotRequiredSinceWeAreLocal",
        base_url="http://0.0.0.0:4000",
        model_capabilities={
            "json_output": False,
            "vision": False,
            "function_calling": True,
        },
    )

In [10]:
# from models import qwen_25_1_5b, llama3_2_3b
# def get_model_client() -> OpenAIChatCompletionClient: 
#     return llama3_2_3b()

In [11]:
@dataclass
class Message:
    content: str

In [12]:
@default_subscription
class Assistant(RoutedAgent):
    def __init__(self, name: str, model_client: ChatCompletionClient) -> None:
        super().__init__("An assistant agent.")
        self._model_client = model_client
        self.name = name
        self.count = 0
        self._system_messages = [
            SystemMessage(
                content=f"Your name is {name} and you are a part of a duo of comedians."
                "You laugh when you find the joke funny, else reply 'I need to go now'.",
            )
        ]
        self._model_context = BufferedChatCompletionContext(buffer_size=5)

    @message_handler
    async def handle_message(self, message: Message, ctx: MessageContext) -> None:
        self.count += 1
        await self._model_context.add_message(UserMessage(content=message.content, source="user"))
        result = await self._model_client.create(self._system_messages + await self._model_context.get_messages())

        print(f"\n{self.name}: {message.content}")

        if "I need to go".lower() in message.content.lower() or self.count > 2:
            return

        await self._model_context.add_message(AssistantMessage(content=result.content, source="assistant"))  # type: ignore
        await self.publish_message(Message(content=result.content), DefaultTopicId())  # type: ignore

In [13]:
runtime = SingleThreadedAgentRuntime()

cathy = await Assistant.register(
    runtime,
    "cathy",
    lambda: Assistant(name="Cathy", model_client=get_model_client()),
)

joe = await Assistant.register(
    runtime,
    "joe",
    lambda: Assistant(name="Joe", model_client=get_model_client()),
)

In [14]:
runtime.start()
await runtime.send_message(
    Message("Joe, tell me a joke."),
    # Message("Joe, 给我讲个笑话。"),
    recipient=AgentId(joe, "default"),
    sender=AgentId(cathy, "default"),
)
await runtime.stop_when_idle()

/var/folders/w0/y_fylk8d1g5474hmb49dt1jr0000gn/T/ipykernel_2613/172301977.py:20: UserWarning: Resolved model mismatch: llama3.2:1b != ollama/llama3.2:1b. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to ollama/llama3.2:1b to enhance token/cost estimation and suppress this warning.
  result = await self._model_client.create(self._system_messages + await self._model_context.get_messages())



Joe: Joe, tell me a joke.

Cathy: (laughs) Oh man, here's one: Why couldn't the bicycle stand up by itself? (pauses for comedic effect) Because it was two-tired! (laughs) I mean, that's just ridiculous. How's that, am I making you laugh or what?

Joe: (laughs) Ahahaha, that's a good one, buddy! I think we're laughing at the absurdity of the situation. The pun on "two-tired" is brilliant. You know, I love it when comedians can take something as mundane as a bicycle and turn it into a hilarious joke. Keep 'em coming!

Cathy: (laughs) Ahahah, nice try, buddy! That joke is a bit "flat" if you know what I mean. But hey, at least you tried to pedal your way into the audience with that one. What do you think? Should we "chain" our next jokes together for some laughs?

Joe: (laughs) Oh man, I'm glad you're enjoying it! I've got another one: Why don't eggs tell jokes? (pauses for comedic effect) Because they'd crack each other up! (laughs) Get it? Crack? Like the shell? Ah, never mind...
